In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

stock_ticker_symbols = []
investment_amounts = []

title = widgets.HTML(value="<h1 style='text-align: center; color: #4CAF50;'>Portfolio Beta Calculator</h1>")

stock_box = widgets.VBox()
add_stock_button = widgets.Button(
    description="Add Stock",
    button_style="success",
    style={'font_weight': 'bold'}
)
calculate_button = widgets.Button(
    description="Calculate Portfolio Beta",
    button_style="primary",
    style={'font_weight': 'bold'}
)
output = widgets.Output()

def add_stock_row(*args):
    ticker = widgets.Text(placeholder="Stock Ticker Symbol", description="Ticker:", layout=widgets.Layout(width='50%'))
    amount = widgets.FloatText(placeholder="Amount Invested", description="Amount:", layout=widgets.Layout(width='50%'))
    stock_ticker_symbols.append(ticker)
    investment_amounts.append(amount)
    stock_box.children = stock_box.children + (widgets.HBox([ticker, amount], layout=widgets.Layout(justify_content='space-between')),)

def fetch_beta(stock, market_index):
    try:
        stock_data = yf.download(stock, period="1y", interval="1d", progress=False)["Adj Close"].dropna()
        market_data = yf.download(market_index, period="1y", interval="1d", progress=False)["Adj Close"].dropna()

        if stock_data.empty or market_data.empty:
            return f"Error: No data found for {stock} or market index."

        df = pd.concat([stock_data, market_data], axis=1, keys=["Stock", "Market"]).dropna()

        if df.empty:
            return f"Error: Insufficient overlapping data for {stock} and market index."

        df["Stock_Returns"] = df["Stock"].pct_change()
        df["Market_Returns"] = df["Market"].pct_change()

        covariance = np.cov(df["Stock_Returns"].iloc[1:], df["Market_Returns"].iloc[1:])[0][1]
        market_variance = np.var(df["Market_Returns"].iloc[1:])
        beta = covariance / market_variance
        return beta
    except Exception as e:
        return f"Error fetching beta for {stock}: {e}"

def calculate_portfolio_beta(*args):
    with output:
        output.clear_output()
        print("Calculating Portfolio Beta...")

        market_index = "^GSPC"

        total_investment = 0
        weighted_betas = []

        for ticker, amount in zip(stock_ticker_symbols, investment_amounts):
            if ticker.value and amount.value > 0:
                beta = fetch_beta(ticker.value, market_index)
                if isinstance(beta, str):
                    print(beta)
                else:
                    weighted_betas.append(beta * amount.value)
                    total_investment += amount.value

        if total_investment > 0:
            portfolio_beta = sum(weighted_betas) / total_investment
            print(f"Portfolio Beta: {portfolio_beta:.2f}")
        else:
            print("No valid stocks or investments to calculate portfolio beta.")


add_stock_button.on_click(add_stock_row)
calculate_button.on_click(calculate_portfolio_beta)

add_stock_row()
display(widgets.VBox([title, stock_box, add_stock_button, calculate_button, output]))